In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Disable tensorflow debugging logs
import json
import tensorflow as tf
import keras_nlp
import tensorflow_text as tf_text
from huggingface_hub import hf_hub_download
from model import GPT
from utils import sample

- Download weights

In [2]:
ckpt_dir = 'openwt_512/best-ckpt'

hf_hub_download(repo_id="milmor/gpt-mini", 
                filename=f"{ckpt_dir}/ckpt-934000.data-00000-of-00001",
                local_dir='./')

hf_hub_download(repo_id="milmor/gpt-mini", 
                filename=f"{ckpt_dir}/ckpt-934000.index",
                local_dir='./')

hf_hub_download(repo_id="milmor/gpt-mini", 
                filename=f"{ckpt_dir}/checkpoint",
                local_dir='./')

config_file = hf_hub_download(repo_id="milmor/mini-gpt", 
                filename="openwt_512/openwt_512_config.json",
                local_dir='./')

In [3]:
with open(config_file) as f:
    config = json.load(f)

In [4]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=config['vocab_file'],
    sequence_length=config['seq_len'] + 1,
    lowercase=False,
)

In [5]:
model = GPT(vocab_size=config['vocab_size'], 
            maxlen=config['seq_len'], emb_dim=config['emb_dim'],
            heads=config['heads'], mlp_dim=config['mlp_dim'],
            depth=config['depth'], rate=config['dropout'], 
            initializer=config['initializer'])

- Initialize the model with a tokenized input

In [6]:
context = 'The magic wolf'
t_context = tokenizer(tf_text.normalize_utf8(context, 'NFKD'))[tf.newaxis, :config['seq_len']]

In [7]:
model(t_context)

<tf.Tensor: shape=(1, 512, 30000), dtype=float32, numpy=
array([[[ 1.24882199e-01, -1.90662801e-01, -5.42290360e-02, ...,
         -1.69452175e-01,  4.31207195e-02,  1.56254560e-01],
        [-3.41758579e-02, -2.29694188e-01, -5.64303249e-02, ...,
         -3.65731306e-02,  4.46557067e-03,  1.43037230e-01],
        [-9.96448100e-03, -1.91211447e-01,  2.62345690e-02, ...,
         -8.47366154e-02, -4.69740480e-05,  1.37593210e-01],
        ...,
        [-1.84914228e-02, -1.91389024e-01,  3.00798025e-02, ...,
         -7.88995177e-02,  2.05447171e-02,  1.02667749e-01],
        [ 7.88632222e-03, -2.08302900e-01,  3.57170664e-02, ...,
         -8.79053846e-02,  3.98159064e-02,  8.13047886e-02],
        [-1.53083075e-02, -1.83072627e-01,  4.67220768e-02, ...,
         -2.98853554e-02,  1.43598095e-02,  1.04361206e-01]]],
      dtype=float32)>

In [8]:
model.summary()

Model: "gpt"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 token_embedding (TokenEmbed  multiple                 7811072   
 ding)                                                           
                                                                 
 dropout_1 (Dropout)         multiple                  0         
                                                                 
 transformer_block (Transfor  multiple                 527104    
 merBlock)                                                       
                                                                 
 transformer_block_1 (Transf  multiple                 527104    
 ormerBlock)                                                     
                                                                 
 transformer_block_2 (Transf  multiple                 527104    
 ormerBlock)                                                   

- Restore weights

In [9]:
ckpt = tf.train.Checkpoint(model=model, step=tf.Variable(0))
ckpt_manager = tf.train.CheckpointManager(ckpt, directory=ckpt_dir, 
                                          max_to_keep=1)
ckpt.restore(ckpt_manager.latest_checkpoint)
print(f'Checkpoint restored from {ckpt_manager.latest_checkpoint} at step {int(ckpt.step)}')

Checkpoint restored from openwt_512/best-ckpt/ckpt-934000 at step 934000


In [10]:
text = sample(model, context, config['seq_len'], config['vocab_file'], k=10)
text

'The magic wolf , the shiny , uninjecting , is an ex - soldier who was in an old , hard and britting shindy . His mother was on an outfield of an old train on the Brix - West train . When he learned he ’ was coming , he learned the secret information , but no one was aware how important , as it may have been and how it would impact a long distance run : It was a tale of an old girl and the girl who took over in her head , in one - off a new ship he has now left to travel in and back for three minutes : her husband and the sister she lives with in a home . The only one of our friends , one with the best son and father and a son is a small brother — but she was born with the girl . In a series called , “ A Girl When It Look Into “ : a . k . an intrylish , a , a little out to me , ” it might also appear like a little intridulity — as a result . This story had become something like The Last Star – A Little Out Boy And a few years before it , I started thinking about this and my mother went